In [65]:
%run 0.0_Dependance.ipynb

In [66]:
df = pd.read_csv("train.csv",index_col=0)
df["N_Year"]=np.round((df['N_Days']/365),2)
df.Age = np.round((df.Age)/365,1)

#label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
colonne_a_encoder = ["Sex","Ascites","Hepatomegaly","Spiders","Edema","Drug","Status"]	
for i in colonne_a_encoder:
    df[i] = le.fit_transform(df[i])

In [67]:
param_grid_initial = {
    # Nombre d'arbres dans le boosting
    'n_estimators': [50, 100, 250, 500, 1000],
    
    # Profondeur de chaque arbre
    'max_depth': [3, 6, 9, 12],
    
    # Taux d'apprentissage (eta)
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    
    # Minimum de perte de réduction nécessaire pour faire une partition supplémentaire sur un noeud de l'arbre
    'gamma': [0, 0.1, 0.5, 1],
    
    # Fraction des observations à être randomisées pour chaque arbre
    'subsample': [0.5, 0.75, 1],
    
    # Fraction de colonnes à être randomisées pour chaque arbre
    'colsample_bytree': [0.5, 0.75, 1],
    
    # Le minimum de poids nécessaire pour une enfant
    'min_child_weight': [1, 5, 10],
    
    # Regularisation L1 sur les poids
    'reg_alpha': [0, 0.1, 1],
    
    # Regularisation L2 sur les poids
    'reg_lambda': [0, 1, 10],
    
    # Choix de la fonction objectif, logloss pour la classification
    'objective': ['multi:softprob'],
    
    # Choix de la stratégie d'évaluation pour le boosting
    'eval_metric': ['mlogloss'],
    
    # Choix du booster
    'booster': ['gbtree'],
}

In [68]:
param_grid_ameliore = {
    # Nombre d'arbres dans le boosting
    'n_estimators': [400,500,600,700,800,900],
    
    # Profondeur de chaque arbre
    'max_depth': [4,5,6],
    
    # Taux d'apprentissage (eta)
    'learning_rate': [0.05,0.06,0.07,0.08,0.09,0.1],
    
    # Minimum de perte de réduction nécessaire pour faire une partition supplémentaire sur un noeud de l'arbre
    'gamma': [0.5,0.7, 1],
    
    # Fraction des observations à être randomisées pour chaque arbre
    'subsample': [0.75, 1],
    
    # Fraction de colonnes à être randomisées pour chaque arbre
    'colsample_bytree': [0.2,0.4,0.5,0.6],
    
    # Le minimum de poids nécessaire pour une enfant
    'min_child_weight': [ 5,6,7,8,9,10],
    
    # Regularisation L1 sur les poids
    'reg_alpha': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1],
    
    # Regularisation L2 sur les poids
    'reg_lambda': [0, 1,2,3,4,5,6,7,8,9,10],
    
    # Choix de la fonction objectif, logloss pour la classification
    'objective': ['multi:softprob'],
    
    # Choix de la stratégie d'évaluation pour le boosting
    'eval_metric': ['mlogloss'],
    
    # Choix du booster
    'booster': ['gbtree'],
}


In [77]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

joblib_path_suivi_metrique_1 = get_output_path_file("df_score_random_v1.joblib")
joblib_path_suivi_metrique_2 = get_output_path_file("df_score_best_test_v1.joblib")

target = df.Status
data = df.drop(columns=["Status","N_Days"])

X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.2, random_state=123)
#Standardisation
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#Modèle instancié
model = XGBClassifier(n_estimators=1000, max_depth=2, learning_rate=0.1, objective="multi:softprob")

randomized_search_CV = RandomizedSearchCV(model, param_distributions=param_grid_ameliore,n_iter=100, cv=5, scoring='neg_log_loss',n_jobs=-1)
df_score_random_v1 = test_randomized_model(joblib_path_suivi_metrique_1, param_grid_ameliore, model, randomized_search_CV)
dump(df_score_random_v1,"df_score_random_v1.joblib")

Récupération du DataFrames existant
Début du randomized: RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                        'eval_metric': ['mlogloss'],
            

['df_score_random_v1.joblib']

In [70]:
dump(df_score_random_v1,"df_score_random_v1.joblib")

['df_score_random_v1.joblib']

In [76]:
df_score_random_v1 = load("df_score_random_v1.joblib")